In [6]:
import os
os.environ["TORCHAUDIO_BACKEND"] = "sox_io"  # sox_io 백엔드를 사용하도록 지정
import torch
import torch.nn as nn
import torch.optim as optim
import torchaudio
from torchaudio.datasets import SPEECHCOMMANDS
from torch.utils.data import DataLoader

# 1. 데이터셋 클래스 정의
# SPEECHCOMMANDS라는 torchaudio의 기본 데이터셋 클래스를 상속받아 SubsetSC라는 새로운 클래스를 정의
# 이 클래스는 데이터셋의 일부분(예: 훈련, 검증, 테스트)을 선택적으로 로드하는 기능을 추가
class SubsetSC(SPEECHCOMMANDS):
    def __init__(self, subset: str = None): # 클래스 생성자 (초기화 메서드), subset 매개변수는 어떤 부분집합을 로드할지 지정하는 문자열
        super().__init__("./data", download=True) # 부모 클래스 SPEECHCOMMANDS의 초기화 메소드를 호출, 데이터셋을 "./data" 디렉터리에 다운로드 및 저장하며, 필요시 자동으로 다운로드하도록 설정
        def load_list(filename): # 주어진 파일 이름을 기반으로 파일 목록을 로드하는 내부 함수 
            filepath = os.path.join(self._path, filename) # filename과 데이터셋의 기본 경로(self._path)를 결합하여 전체 파일 경로 설정 
            if not os.path.exists(filepath): # 지정한 경로에 파일이 존재하지 않으면 경고 메시지를 출력하고, 빈 리스트를 반환. 즉, 파일이 없으면 건너 뜀
                print(f"File {filepath} not found. Skipping...")
                return []
            with open(filepath) as f: # 파일이 존재하면 해당 파일을 열고, 각 줄을 읽음 
                return [os.path.join(self._path, line.strip()) for line in f]
        if subset == "validation": # subset 매개변수가 "validation"이면, validation_list.txt 파일을 로드하여 _walker에 저장
            self._walker = load_list("validation_list.txt")
        elif subset == "testing": # subset이 "testing"인 경우,testing_list.txt를 로드하여 _walker에 저장
            self._walker = load_list("testing_list.txt")
        elif subset == "training": # subset이 "training"인 경우, 검증(validation)과 테스트(testing) 파일 목록을 로드하여 하나의 리스트로 합침
            excludes = load_list("validation_list.txt") + load_list("testing_list.txt") # 검증 파일 + 평가 파일을 excludes에 저장
            excludes = set(excludes) # 이 목록을 set으로 변환하여 중복 제거 및 빠른 조회가 가능하도록 수정
            self._walker = [w for w in self._walker if w not in excludes] # excludes에 있는 파일 제외한 파일만 _walker

# 요약:
# SubsetSC 클래스는 SpeechCommands 데이터셋의 특정 부분집합을 로드할 수 있게 해줍니다.
# subset 매개변수에 따라 검증, 테스트 또는 훈련 데이터 파일 목록을 _walker에 설정합니다.
# 훈련 데이터셋의 경우, 검증과 테스트 파일 목록을 제외한 나머지 파일들을 선택합니다.



# 2. Transformer 기반 음성 명령 인식 모델 정의
#   -------------------------------------------------------------------
#    input_dim: 입력 특성 차원 (예: Mel-spectrogram의 멜 밴드 수)            - 128차원
#    num_classes: 분류할 클래스의 개수.
#    d_model: Transformer 내부의 임베딩 차원.                              - 128 차원 
#    nhead: 멀티헤드 어텐션에서의 헤드 수.                                  - 8개 
#    num_layers: Transformer 인코더 레이어의 수.                            - 4개
#    dim_feedforward: 각 Transformer 레이어 내 피드포워드 네트워크의 차원. - 512차원
#    dropout: 드롭아웃 비율.
#   -------------------------------------------------------------------
class SpeechTransformer(nn.Module):
    def __init__(self, input_dim, num_classes, d_model=128, nhead=8, num_layers=4, dim_feedforward=512, dropout=0.1):
        super(SpeechTransformer, self).__init__()
        self.input_projection = nn.Linear(input_dim, d_model) # 입력 특성 차원(input_dim)을 Transformer 모델에서 사용하는 차원(d_model)으로 선형 변환하는 레이어
        # 단일 Transformer 인코더 레이어를 생성. encoder_layer는 init 메서드 외에서는 사용되지 않으므로 지역 변수로 생성함
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, 
            nhead=nhead, 
            dim_feedforward=dim_feedforward, 
            dropout=dropout
        )
        # 단일 Transformer 인코더를 num_layers만큼 쌓아서 
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.classifier = nn.Linear(d_model, num_classes)
        
    def forward(self, x):
        # x shape: (batch_size, seq_len, input_dim)
        #print (x)
        out = self.input_projection(x)             # -> (batch_size, seq_len, d_model)
        #print(out)
        out = out.transpose(0, 1)                    # Transformer 입력 shape: (seq_len, batch_size, d_model)
        out = self.transformer_encoder(out)          # -> (seq_len, batch_size, d_model)
        #print(out)
        out = out.mean(dim=0)                        # 시퀀스 차원 평균 -> (batch_size, d_model)
        #print(out)
        logits = self.classifier(out)              # -> (batch_size, num_classes)
        #print(logits)
        return logits
    





# 3. 평가 함수 정의
def evaluate(model, data_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in data_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, dim=1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
    return correct / total

# 4. main 함수 정의
def main():
    transform = torchaudio.transforms.MelSpectrogram(sample_rate=16000, n_mels=128)

    # 데이터셋 로드
    train_set = SubsetSC("training")
    test_set = SubsetSC("testing")

    # 라벨 목록 및 인덱스 매핑 생성
    labels = sorted(list(set(dat[2] for dat in train_set)))
    label_to_idx = {label: idx for idx, label in enumerate(labels)}

    def collate_fn(batch):
        waveforms = []
        targets = []
        for waveform, sample_rate, label, *_ in batch:
            mel_spec = transform(waveform)
            mel_spec = mel_spec.mean(dim=0)
            mel_spec = mel_spec.transpose(0, 1)
            waveforms.append(mel_spec)
            targets.append(label_to_idx[label])
        waveforms = nn.utils.rnn.pad_sequence(waveforms, batch_first=True)
        targets = torch.tensor(targets)
        return waveforms, targets

    train_loader = DataLoader(train_set, batch_size=32, shuffle=True, collate_fn=collate_fn)
    test_loader = DataLoader(test_set, batch_size=32, shuffle=False, collate_fn=collate_fn)

    input_dim = 128
    num_classes = len(labels)
    model = SpeechTransformer(input_dim=input_dim, num_classes=num_classes)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-3)

    num_epochs = 10
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        avg_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")

    accuracy = evaluate(model, test_loader)
    print(f"Test Accuracy: {accuracy * 100:.2f}%")

if __name__ == '__main__':
    main()


c:\Users\Admin\anaconda3\envs\stt_env\lib\site-packages\torch\nn\modules\transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


Epoch 1/10, Loss: 1.9380
Epoch 2/10, Loss: 1.2769
Epoch 3/10, Loss: 1.0876
Epoch 4/10, Loss: 0.9995
Epoch 5/10, Loss: 0.9530
Epoch 6/10, Loss: 0.8923
Epoch 7/10, Loss: 0.8470
Epoch 8/10, Loss: 0.8180
Epoch 9/10, Loss: 0.7867
Epoch 10/10, Loss: 0.7556
Test Accuracy: 80.36%
